Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re  #regular expression is useful for going through the text in a particlar paragraph or a document.
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')  # stopwords are those words which are repeated a lot of time in paragraph or a document.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

**Data Pre-Processing**

In [ ]:
# load the data to a pandas dataframe
news_data = pd.read_csv('/content/train.csv', engine='python', quotechar='"', escapechar='\\', on_bad_lines='skip')

In [ ]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


0--> real news

1--> fake news

In [ ]:
news_data.shape

(9883, 5)

In [ ]:
# checking for missing values
news_data.isnull().sum()

,0
id,0
title,269
author,934
text,21
label,0


In [ ]:
#replacing the missing values with null string
news_data=news_data.fillna('')

In [ ]:
#merging the author name and news title
news_data['content']=news_data['author']+' '+news_data['title']

In [ ]:
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
# sepersting feature and target
x=news_data.drop(columns='label',axis=1)
y=news_data['label']

In [ ]:
print(x)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
6429  6429  Moveable Feast Cafe 2016/10/28 … Open Thread |...   
6430  6430  Hajj Prep: Search Soul, Buy Sturdy Shoes, Pay ...   
6431  6431  10 radikale Vorhaben, die Donald Trump sofort ...   
6432  6432                               Little Blue Balls...   
6433  6433  WATCH: Louis CK’s EPIC Answer on Trump vs. Hil...   

                                   author  \
0                           Darrell Lucus   
1                         Daniel J. Flynn   
2                      Consortiumnews.com   
3                         Jessica Purki

In [ ]:
print(y)

0       1
1       0
2       1
3       1
4       1
       ..
6429    1
6430    0
6431    1
6432    1
6433    1
Name: label, Length: 6434, dtype: int64


Stemming:

Stemming is the process of reducing a word to its Root Word.

In [ ]:
 port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_data['content']=news_data['content'].astype(str).apply(stemming)

In [ ]:
print(news_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20816    jerom hudson rapper trump poster child white s...
20817    benjamin hoffman n f l playoff schedul matchup...
20818    michael j de la merc rachel abram maci said re...
20819    alex ansari nato russia hold parallel exercis ...
20820                            david swanson keep f aliv
Name: content, Length: 20821, dtype: object


In [ ]:
x=news_data['content'].values
y=news_data['label'].values

In [ ]:
print(x)

[list(['darrel', 'lucu', 'hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet'])
 list(['daniel', 'j', 'flynn', 'flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart'])
 list(['consortiumnew', 'com', 'truth', 'might', 'get', 'fire']) ...
 list(['norepli', 'blogger', 'com', 'der', 'postillon', 'radikal', 'vorhaben', 'die', 'donald', 'trump', 'sofort', 'nach', 'amtsantritt', 'durchsetzen', 'wird'])
 list(['recrisp', 'littl', 'blue', 'ball'])
 list(['grant', 'stern', 'watch', 'loui', 'ck', 'epic', 'answer', 'trump', 'vs', 'hillari'])]


In [ ]:
print(y)

[1 0 1 ... 1 1 1]


In [ ]:
y.shape

(6434,)

In [ ]:
x.shape

(6434,)

In [ ]:
# converting the textual data to feature vectors
vectorizer=TfidfVectorizer()
vectorizer.fit(x)  # we are fitting all the data in content to this vectorizer funtion.
x=vectorizer.transform(x)

In [ ]:
print(x)

  (0, 267)	0.2701476206362224
  (0, 2484)	0.3676995795693613
  (0, 2960)	0.24689108808738355
  (0, 3602)	0.359941431098289
  (0, 3794)	0.27057962443509487
  (0, 4975)	0.23336287017959056
  (0, 7008)	0.21878740888803716
  (0, 7695)	0.2478982832436769
  (0, 8633)	0.29217179168954643
  (0, 8912)	0.3636439399690358
  (0, 13477)	0.2560146587984509
  (0, 15694)	0.2848971939754382
  (1, 1497)	0.29399205102022696
  (1, 1894)	0.1552451057257906
  (1, 2224)	0.3827205648196221
  (1, 2814)	0.19096531956187138
  (1, 3570)	0.2637454809945214
  (1, 5506)	0.7143152671504139
  (1, 6819)	0.19048567644127162
  (1, 16806)	0.3007192619627194
  (2, 2944)	0.3180042868355105
  (2, 3104)	0.46096698415377874
  (2, 5392)	0.38665736793549876
  (2, 5971)	0.34747209776436094
  (2, 9623)	0.49350166555600666
  :	:
  (20818, 3645)	0.21156290306650766
  (20818, 7045)	0.2179974648085888
  (20818, 8367)	0.22323208520517143
  (20818, 8991)	0.36159510927678556
  (20818, 9521)	0.2954162937577498
  (20818, 9591)	0.1745666731